# Initialization

In [16]:
#import *
import numpy as np
import os
import nglview as ngl
from ase.io import read, write
WDIR='/home/harish/GroPolBul/tutorial/'
#%mkdir {WDIR}
%cd {WDIR}

#Desired polymer electrolyte system
#%mkdir PEO_LiTFSI
%cd PEO_LiTFSI

/home/harish/GroPolBul/tutorial
/home/harish/GroPolBul/tutorial/PEO_LiTFSI


## Initial ITP and topology files

The required *.itp files are added to GroProBul/ITP folder, which are then used to generate topology files for desired PEO_LITFSI system.

In [11]:
### Make the topology file with ITP directory location and system details
ITPDIR='/home/harish/GroPolBul/tutorial/ITP'
npol=40;nmon=25;conc=0.08        
nions=npol*nmon*conc
topol=open('topol.top','w+')
topol.write('''#include "'''+str(ITPDIR)+'''/ff.itp"
#include "'''+str(ITPDIR)+'''/ITP/PEO_'''+str(nmon)+'''mer.itp"    
#include "'''+str(ITPDIR)+'''/ITP/li_75c.itp"  
#include "'''+str(ITPDIR)+'''/ITP/tfsi_75c.itp"
        
[ system ] 
PEO_LiTFSI_'''+str(conc)+'''

[ molecules ]
polymer   '''+str(npol)+'''     
LI    '''+str(nions)+'''
TFS   '''+str(nions)+'''
''')
topol.close() 


## Simulation boxes (packmol)

In [14]:
#Build initial simulation boxes using packmol
pack=open('packmol.inp','w+')
pack.write('''tolerance 2.0
filetype pdb
output initial.pdb
structure PEO_25mer.pdb
  number '''+str(npol)+'''
  inside cube 0. 0. 0. 100. 
end structure

structure li.pdb
  number '''+str(nions)+'''
  inside cube 20. 20. 20. 50.
end structure

structure tfsi.pdb
  number '''+str(nions)+'''
  inside cube 20. 20. 20. 50.
end structure
        ''')
pack.close()
!packmol < packmol.inp



################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.010 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  ERROR: Error reading number of molecules of type            2


In [18]:
vi=ngl.show_ase(read('initial.pdb'),viewer='ngl')
vi

NGLWidget()

# MD simulations (Gromacs)

## Energy minimization

In [21]:
%%bash
#RUN in beskow/tegner terminal for preparing initial configuration file and EM step

gmx editconf -f initial.pdb -o preem.gro -c -bt cubic -d 0
mkdir em
cd em
echo '''; em.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
dt          = 0.001	    ; ps
emtol       = 100.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 100000          ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
rlist           = 1.3
cutoff-scheme   = Verlet    ; Buffered neighbor searching
;coulombtype    = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.3       ; Short-range electrostatic cut-off
vdwtype         = cut-off
rvdw            = 1.3       ; Short-range Van der Waals cut-off
fourierspacing  = 0.16
;pme_order      = 4
ewald_rtol      = 1e-4
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
''' > em.mdp

gmx grompp -f em.mdp -c ../preem.gro -p ../topol.top -o em.tpr -maxwarn 2
gmx mdrun -s em.tpr -deffnm em -v
cd ..


Process is interrupted.


In [19]:
vi=ngl.show_ase(read('em/em.gro'),viewer='ngl')
vi

NGLWidget()

 ## Pre-equilibration step

In [ ]:
%%bash
#RUN for preeq step

mkdir preeq
cd preeq

#Initial NVT 
mdp=/home/harish/GroPolBul/mdp/nvt_preeq.mdp
gmx grompp -f $mdp -c ../em/em.gro -p ../topol.top -o nvt.tpr -maxwarn 2
gmx mdrun -s nvt.tpr -cpi nvt.cpt -deffnm nvt -v

#Initial NPT
mdp=/home/harish/GroPolBul/mdp/nvt_preeq.mdp
gmx grompp -f $mdp -c nvt.gro -p ../topol.top -o npt.tpr -maxwarn 2
gmx mdrun -s npt.tpr -cpi npt.cpt -deffnm npt -v

cd ..

In [20]:
vi=ngl.show_ase(read('preeq/nvt.gro'),viewer='ngl')
vi

NGLWidget()

In [21]:
vi=ngl.show_ase(read('preeq/npt.gro'),viewer='ngl')
vi

NGLWidget()

## NPT Equilibiriation step

In [22]:
%%bash
#RUN for NPT equilibiration step at a desired initial temperature 400K

mkdir 400
cd 400

mdp=/home/harish/GroPolBul/mdp/npt_eq_T.mdp
gmx grompp -f $mdp -c ../preeq/npt.gro -p ../topol.top -o npt_eq.tpr -maxwarn 2
gmx mdrun -s npt_eq.tpr -cpi npt_eq.cpt -deffnm npt_eq -v

cd ..

mkdir: cannot create directory ‘400’: File exists
gmx: error while loading shared libraries: libopenblas.so.0: cannot open shared object file: No such file or directory
gmx: error while loading shared libraries: libopenblas.so.0: cannot open shared object file: No such file or directory


In [23]:
vi=ngl.show_ase(read('400/npt_eq.gro'),viewer='ngl')
vi

NGLWidget()